In [1]:
import pandas as pd
import string
import datetime
from datetime import datetime
from datetime import timedelta
import numpy as np
import os

from random import random

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_tournois=pd.read_csv('..\\raw_data\\tournaments_1877-2017_unindexed_csv.csv')
df_tournois=df_tournois[df_tournois['tourney_year']>=1991]
dict_tournoi=df_tournois[['tourney_year','tourney_dates','tourney_slug','tourney_name','tourney_conditions','tourney_location','tourney_singles_draw','tourney_surface']]
dict_tournoi['tourney_dates']=pd.to_datetime(dict_tournoi['tourney_dates'])

#We don't have data for years from 2018 to 2020, let's assume that tournament haven't change in these 3 years
#We can then add lines corresponding to tournaments from 2018 to 2020
tournois_2017=dict_tournoi[dict_tournoi['tourney_year']==2017]
k=1
for year in range (2018,2021):
    df_year=tournois_2017
    df_year['tourney_year']=year
    df_year['tourney_dates']=pd.to_datetime(df_year['tourney_dates'])
    df_year['tourney_dates']=df_year['tourney_dates']+timedelta(days=365)
    k+=1
    dict_tournoi=pd.concat([dict_tournoi,df_year])
dict_tournoi['tourney_dates']=dict_tournoi['tourney_dates'].apply(lambda x:x.strftime('%Y-%m-%d'))

#Create a function to avoid problems with location formatin over time
def change_loc(location):
    location=str(location).split(',')[0]
    return location

# just take the city name if the location format is city, country
dict_tournoi['tourney_location']=dict_tournoi['tourney_location'].apply(lambda x:change_loc(x))
# create a code for each tournament instance
dict_tournoi['tournoi_unique_id']=dict_tournoi['tourney_year'].astype(str)+'-'+dict_tournoi['tourney_location'].apply(lambda x:str(x).lower())+'-'+dict_tournoi['tourney_surface'].apply(lambda x:str(x).lower())
# create a list of duplicate tournament codes
#list_not_unique_2=dict_tournoi['tournoi_unique_id'].value_counts()[dict_tournoi['tournoi_unique_id'].value_counts()>1]
# create another code for each tournament instance
dict_tournoi['unique_slug']=dict_tournoi['tourney_year'].apply(lambda x:str(x))+'-'+dict_tournoi['tourney_slug']
# drop the generic slug column
dict_tournoi=dict_tournoi.drop('tourney_slug',axis=1).reset_index(drop=True)

#Here some 'tournoi_unique_id' are not unique, mainly because of Queens and wimbledon
list_not_unique=dict_tournoi['tournoi_unique_id'].value_counts()[dict_tournoi['tournoi_unique_id'].value_counts()>1]
dict_tournoi.loc[dict_tournoi['tournoi_unique_id'].isin(list_not_unique.index),'tournoi_unique_id']=dict_tournoi.loc[dict_tournoi['tournoi_unique_id'].isin(list_not_unique.index)]['tournoi_unique_id']+'-'+dict_tournoi.loc[dict_tournoi['tournoi_unique_id'].isin(list_not_unique.index)]['tourney_singles_draw'].apply(lambda x:str(x))
list_not_unique_2=dict_tournoi['tournoi_unique_id'].value_counts()[dict_tournoi['tournoi_unique_id'].value_counts()>1]
dict_tournoi=dict_tournoi[~dict_tournoi['tournoi_unique_id'].isin(list_not_unique_2.index)]

# add a binary column with 1 for major tournaments
majors = ["Wimbledon", "Roland Garros", "US Open", "Australian Open"]
dict_tournoi['d_major'] = np.zeros(dict_tournoi.shape[0])
dict_tournoi.loc[dict_tournoi['tourney_name'].isin(majors),'d_major'] = 1

In [3]:
dict_tournoi.to_csv("processed_data\\tournaments_clean.csv")